# Information Retrieval for NBA subreddit reviews

In [ ]:
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-08 17:43:33.885727: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


## Build a search engine on the Reddit data 

In [14]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [15]:
import pandas as pd
data = pd.read_csv('/gdrive/My Drive/DecisionTree/Reddit.nba/output_nba (2).csv')
data

,MsgID,Timestamp,Author,ThreadID,ThreadTitle,MsgBody,ReplyTo,Permalink
0,'j7s0r0z','2023-02-09 00:08:34','REQ52767','10xfakf','What would you think about LeBron getting tra...,'The Clippers are above you in the standings lol','j7s0kfx','/r/nba/comments/10xfakf/what_would_you_think_...
1,'j7s0r2r','2023-02-09 00:08:35','Methzilla','10x9u1k','Kareem Abdul-Jabbar Is Greater Than Any Baske...,'Hard to say. Then the nba miles on his body s...,'j7rmsby','/r/nba/comments/10x9u1k/kareem_abduljabbar_is...
2,'j7s0rkk','2023-02-09 00:08:40','unexpectedvillain','10xcwdd','[Wojnarowski] ESPN Sources: Three-team trade ...,'Conley stats are 10/7 DLo is 18/6:','j7rvies','/r/nba/comments/10xcwdd/wojnarowski_espn_sour...
3,'j7s0rnc','2023-02-09 00:08:41','SuperElection3215','10xf5v9','Mavericks Began Planning Pursuit Of Kyrie Irv...,'They not gonna win anything this year the mav...,'-','/r/nba/comments/10xf5v9/mavericks_began_plann...
4,'j7s0rpy','2023-02-09 00:08:42','OkSteak237','10xfc1b','I was surprised to see that jordan has better...,'We all know why. People can’t accept greatnes...,'j7s0btu','/r/nba/comments/10xfc1b/i_was_surprised_to_se...
...,...,...,...,...,...,...,...,...
108,'j7s1di6','2023-02-09 00:12:59','ender23','10xff6g','[Charania] “For the Lakers this would be turn...,'Only if the jazz and wolves are dumb. And we...,'-','/r/nba/comments/10xff6g/charania_for_the_lake...
109,'j7s1dkn','2023-02-09 00:12:59','Vordeo','10xff6g','[Charania] “For the Lakers this would be turn...,'> Conley doesn’t have value with that contrac...,'j7s0s7i','/r/nba/comments/10xff6g/charania_for_the_lake...
110,'j7s1do7','2023-02-09 00:13:00','shaheimjay1121','10xe1fj','3 team deal between Lakers T Wolves & Jazz','I swear any other team they would say this is...,'j7rwf79','/r/nba/comments/10xe1fj/3_team_deal_between_l...
111,'j7s1dv2','2023-02-09 00:13:03','SputnikFace','10x5985','Brooks on getting booed in Memphis: “I get bo...,'Brooks who?!?! Brooks Hatlen!?','-','/r/nba/comments/10x5985/brooks_on_getting_boo...


In [16]:
data.head()

,MsgID,Timestamp,Author,ThreadID,ThreadTitle,MsgBody,ReplyTo,Permalink
0,'j7s0r0z','2023-02-09 00:08:34','REQ52767','10xfakf','What would you think about LeBron getting tra...,'The Clippers are above you in the standings lol','j7s0kfx','/r/nba/comments/10xfakf/what_would_you_think_...
1,'j7s0r2r','2023-02-09 00:08:35','Methzilla','10x9u1k','Kareem Abdul-Jabbar Is Greater Than Any Baske...,'Hard to say. Then the nba miles on his body s...,'j7rmsby','/r/nba/comments/10x9u1k/kareem_abduljabbar_is...
2,'j7s0rkk','2023-02-09 00:08:40','unexpectedvillain','10xcwdd','[Wojnarowski] ESPN Sources: Three-team trade ...,'Conley stats are 10/7 DLo is 18/6:','j7rvies','/r/nba/comments/10xcwdd/wojnarowski_espn_sour...
3,'j7s0rnc','2023-02-09 00:08:41','SuperElection3215','10xf5v9','Mavericks Began Planning Pursuit Of Kyrie Irv...,'They not gonna win anything this year the mav...,'-','/r/nba/comments/10xf5v9/mavericks_began_plann...
4,'j7s0rpy','2023-02-09 00:08:42','OkSteak237','10xfc1b','I was surprised to see that jordan has better...,'We all know why. People can’t accept greatnes...,'j7s0btu','/r/nba/comments/10xfc1b/i_was_surprised_to_se...


In [17]:
messages = data['MsgBody']

In [19]:
import spacy.cli

spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [21]:
import spacy #load spacy
nlp = spacy.load("en_core_web_lg")
def normalize(review, lowercase, remove_stopwords):
    if lowercase:
        review = str(review).lower()
    doc = nlp(review)
    lemmatized = list()
    for token in doc:
        if not remove_stopwords or (remove_stopwords and not token.is_stop):
            lemmatized.append(token.lemma_)
    return " ".join(lemmatized)
data['processed'] = data['MsgBody'].apply(normalize, lowercase=True, remove_stopwords=True)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
vectorizer = TfidfVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize, use_idf=True, smooth_idf=True, sublinear_tf=True, norm='l2')
tf_idf_mx = vectorizer.fit_transform(data['processed'])


In [24]:
print(tf_idf_mx)

  (0, 370)	0.4840845938990738
  (0, 594)	0.6440838353714465
  (0, 121)	0.5922990114484206
  (1, 297)	0.27927659529701676
  (1, 242)	0.25682254736249954
  (1, 394)	0.24089114681641272
  (1, 196)	0.24089114681641272
  (1, 617)	0.27927659529701676
  (1, 586)	0.47285637992352486
  (1, 597)	0.40786416605406106
  (1, 90)	0.27927659529701676
  (1, 402)	0.27927659529701676
  (1, 418)	0.24089114681641272
  (1, 274)	0.24089114681641272
  (2, 33)	0.41419779369859555
  (2, 5)	0.41419779369859555
  (2, 184)	0.32396615377876425
  (2, 34)	0.3808959801177319
  (2, 1)	0.41419779369859555
  (2, 600)	0.3808959801177319
  (2, 138)	0.3003381410206603
  (3, 245)	0.23118501455544033
  (3, 574)	0.34924265994803483
  (3, 76)	0.32116328788711634
  (3, 544)	0.32116328788711634
  :	:
  (112, 469)	0.18813884085954855
  (112, 220)	0.18813884085954855
  (112, 447)	0.18813884085954855
  (112, 54)	0.18813884085954855
  (112, 142)	0.18813884085954855
  (112, 636)	0.18813884085954855
  (112, 168)	0.18813884085954855
  (

In [25]:
doc_vec_mat = pd.DataFrame(tf_idf_mx.toarray(), columns = vectorizer.get_feature_names_out(), index = data['MsgID'])
print(doc_vec_mat)

                  1        10  10th        14   15        18  1992  1st    2  \
MsgID                                                                          
'j7s0r0z'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s0r2r'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s0rkk'  0.000000  0.414198   0.0  0.000000  0.0  0.414198   0.0  0.0  0.0   
'j7s0rnc'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s0rpy'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
...             ...       ...   ...       ...  ...       ...   ...  ...  ...   
'j7s1di6'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s1dkn'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s1do7'  0.207356  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s1dv2'  0.000000  0.000000   0.0  0.000000  0.0  0.000000   0.0  0.0  0.0   
'j7s1e68'  0.000000  0.000000   0.0  0.1

In [26]:
query_1 = "lebron" 
query_2 = "win"
query_3 = 'game'
query_vec = vectorizer.transform([query_1, query_2, query_3])
query_vec_mat = pd.DataFrame(query_vec.toarray(), columns = vectorizer.get_feature_names_out(), index = ["q1", "q2", "q3"])
print(query_vec_mat)

      1   10  10th   14   15   18  1992  1st    2   20  ...  work  worth  \
q1  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  ...   0.0    0.0   
q2  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  ...   0.0    0.0   
q3  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  ...   0.0    0.0   

    wrong  www    y  yea  yeah  year  yes   yr  
q1    0.0  0.0  0.0  0.0   0.0   0.0  0.0  0.0  
q2    0.0  0.0  0.0  0.0   0.0   0.0  0.0  0.0  
q3    0.0  0.0  0.0  0.0   0.0   0.0  0.0  0.0  

[3 rows x 691 columns]


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
results = pd.DataFrame(cosine_similarity(doc_vec_mat, query_vec_mat), columns=["Q1", 
"Q2", "Q3"], index = data['MsgBody'])
print(results)

                                                     Q1        Q2        Q3
MsgBody                                                                    
'The Clippers are above you in the standings lol'   0.0  0.000000  0.000000
'Hard to say. Then the nba miles on his body st...  0.0  0.000000  0.000000
'Conley stats are 10/7  DLo is 18/6:'               0.0  0.000000  0.000000
'They not gonna win anything this year the mavs...  0.0  0.273161  0.231185
'We all know why. People can’t accept greatness...  0.0  0.000000  0.000000
...                                                 ...       ...       ...
'Only if the jazz and wolves are dumb.  And we ...  0.0  0.000000  0.000000
'> Conley doesn’t have value with that contract...  0.0  0.000000  0.226016
'I swear any other team they would say this isn...  0.0  0.000000  0.000000
'Brooks who?!?! Brooks Hatlen!?'                    0.0  0.000000  0.000000
'Conleys contract is partially guaranteed for 1...  0.0  0.000000  0.000000

[113 rows x

In [30]:
results.sort_values(by="Q2", ascending=False)


,Q1,Q2,Q3
MsgBody,,,
'Kyrie one of the best finishers around the rim ever. One of the best clutch scorers and performed under pressure in the playoffs. Had the best handles in the league when they won. Win MVP of all star game 2014. Love best rebounder and one of the best outlet passers in the league when they won. Wade best scorer in the league when they won. AD was the best PF and small ball center when they won. He gets injured but he didnt when they won a championship. I have to bring up your comment on Bosh acting like he was just some all-star role player. Check his career stats. A lot of people felt he should have been in top 75. Guys was a franchise number 1 for almost a decade before Mia.',0.000000,0.298026,0.090348
'They not gonna win anything this year the mavs squad around them is too weak for a deep run. But they gonna ball out and be showtime every game',0.000000,0.273161,0.231185
'KD ruined his own legacy by partnering with such a shithead when he could’ve won 4 championships right now with the warriors.',0.000000,0.268205,0.000000
'Celtics starters tonight: - Blake Griffin - Grant Williams - Jayson Tatum - Jaylen Brown - Derrick White 76ers starters: - Joel Embiid - P.J. Tucker - Tobias Harris - DeAnthony Melton - James Harden Sixers should win this with the Cs missing 3 starters',0.000000,0.139701,0.000000
'This is why the total points record is kinda corny. Is emmitt the greatest RB of all time? Of course not and nobody is seriously going to debate it. Jordan went to college for 3 years retired for 2 seasons in the middle of two 3peats and then retired again at like 36 for a few years. He still scored like 32k points. If you look at total points per season lebrons top season would be Jordans 10th. Jordan play all 82 games more than half the seasons he played. LeBron has done it once in 20 seasons. LeBron managed his games and minutes to be able to compete at a higher level longer. Jordan worked like a psychopath to win as many titles as he could as fast as he could. Jordan played every game after he came back from baseball and 3 peated again. He literally didnt miss one game over 3 seasons. LeBron is insane but anyone that watched both players throughout their entire careers will tell you Jordan was better and its not even really close. Jordan is the greatest scorer in NBA history. Theres a damn good argument that he is a top 3 defender at SG. His 86-87 season is one that rivals wilts stuff. DPOY and averaged fucking 37 PPG 😂',0.176248,0.079669,0.160899
...,...,...,...
'Lol a second? Thats insane.',0.000000,0.000000,0.000000
'I’ll happily oblige and boo you since you’ve made it clear to everyone that you care waaaaaaay too much about what everyone thinks. People who don’t care don’t respond. lol',0.000000,0.000000,0.000000
'Neto career high incoming',0.000000,0.000000,0.000000
